# Project 8 -- Address Book

[Faculty Staff Directory](https://pharmacy.buffalo.edu/faculty-staff.html)

In [1]:
import requests
import re
from time import time

In [2]:
facultyUrl = 'https://pharmacy.buffalo.edu/faculty-staff.html'
facultyHTML = requests.get(facultyUrl).text

In [3]:
facultyHTML = facultyHTML.replace('>','>\n')

In [4]:
#print(facultyHTML)

In [5]:
start = '''<div class="text_area">
<h3>
<a name="a">
A</a>
</h3>
<div class="facultyindexsection">
<div class="ecvpage page unstructuredpage">
'''

end = '''<div class="bottom_container">
</div>
<!--end bottom container-->
</div>
'''

In [6]:
pharmDirectory = facultyHTML.split(start)[1].split(end)[0] 

In [7]:
for pharm in pharmDirectory.split('<div>\n'):
    print(pharm)
    print('-'*50)


--------------------------------------------------
<a class="facultyindexname" href="https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4">
Adams, Kimberly</a>
</div>
</div>
<div class="ecvpage page unstructuredpage">

--------------------------------------------------
<a class="facultyindexname" href="https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=adolf">
Adolf, Maureen</a>
</div>
</div>
<div class="ecvpage page unstructuredpage">

--------------------------------------------------
<a class="facultyindexname" href="https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=manujahu">
Ahuja, Manuj</a>
</div>
</div>
<div class="ecvpage page unstructuredpage">

--------------------------------------------------
<a class="facultyindexname" href="https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=npaolini">
Albanese, Nicole, PharmD</a>
</div>
</div>

In [8]:
pharmDirectory_list = [pharm for pharm in pharmDirectory.split('<div>\n')]

In [9]:
urlPattern = r'href=(\".+\"){1}'

In [10]:
pharmDirectoryURLS_list = [re.findall(urlPattern,url) for url in pharmDirectory_list]

In [11]:
pharmDirectoryURLS_list

[[],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=adolf"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=manujahu"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=npaolini"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=attwood3"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=jbalthas"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=jb"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=svb"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=bsbasile"'],
 ['"https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.htm

In [12]:
del(pharmDirectoryURLS_list[0]) # This contained an empty value, delete

We need to now remove all `"` characters from every string in `pharmDirectoryURLS_list` in order for `requests.get()` to be able to properly work.

In [13]:
for url in pharmDirectoryURLS_list:
    url[0] = url[0].replace('"','')

In [14]:
pharmDirectoryURLS_list

[['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=kadams4'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=adolf'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=manujahu'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=npaolini'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=attwood3'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=jbalthas'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=jb'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=svb'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=bsbasile'],
 ['https://pharmacy.buffalo.edu/content/pharmacy/faculty-staff/faculty-profile.html?ubit=eb'],
 ['https://

Now with our url's properly formatted, we can use `requests.get()` to pull the html from every graduate's webpage.

Looking for where the information that we need is stored, it seems that every faculty member's name is stored between a `<span>` named `facultyprofilecontainer` and a `<span>` named `facultyprofiledegrees`. Once we have the name of the faculty member, we need to locate their contact/office information. This is found between a `<div>` named `facultyprofilemobilecontact` and one named `facultyprofilebreaker`. 

Now that we have the html for each faculty member's page, we want to split it at these elements, which will decrease the information that we have to look through to find what we are looking for significantly.

In [15]:
name_start = '''<span id="facultyprofilecontainer">
<div class="facultyprofilecontainer">
<div class="facultyprofilebody">
<div class="facultyprofilebodytop">
<div class="title nonmobile">
<h1>
'''
name_end = '''<span class="facultyprofiledegrees">
 <span id="lblFacultySuffix">
</span>
</span>
 </h1>
</div>
'''

contactInfo_start = '''<div class="facultyprofilemobilecontact facultyprofile-show-in-mobile">
<div class="facultyprofilecontact">
<a href="#" class="facultyprofilemobile-expand">
<span class="title">
Contact Information<span>
</a>
'''

contactInfo_end = '''<div class="facultyprofilebreaker">
<hr class="facultyprofilehorizonalbreaker"/>
</div>
'''

Regex to pattern match each name, address, phone number, and email.

In [16]:
namePattern = r'<span\sid=\"lblFacultyName\">\n(\w+.+)<'

addressPattern = r'<span id=\"lblAcademicOfficeAddressMobile\">\n((?:.|\n)*)<br/>\nPhone:'

phonePattern = r'Phone:.((?:\(\d{3}\) ){,1}\d{3}-\d{4})'

emailPattern = r'<a\shref=\"mailto:(\w+@buffalo.edu)"'

In [17]:
facultyInfo = []

Most of the phone numbers listed under the faculty member's profile page are in different formats, so we want to correctly format them before we store them in our `facultyInfo` list. This function uses `re.match()` to find different patterns of phone numbers. `re.match()` can be interpretad as a bool, so we can use `if-else` statements to find different formats of phone numbers.

In [18]:
def formatPhoneNumber(phone=str):
    if bool(re.match(r'^\d{7}$',phone)): # 6456112
        phone = '(716)' + phone[0:3] + '-' + phone[3:]
    elif bool(re.match(r'^\d{3}-\d{4}$',phone)): # 645-6112
        phone = '(716)' + phone
    elif bool(re.match(r'^\d{10}$',phone)): # 7166456112
        phone = '(' + phone[0:3] + ')' + phone[3:6] + '-' + phone[6:]
    elif bool(re.match(r'^\d{3}-\d{3}-\d{4}$',phone)): # 716-645-6112
        phone = '(' + phone[0:3] + ')' + phone[4:] 
    elif bool(re.match(r'^\(\d*\)\d{7}$',phone)): # (716)6456112
        phone = phone[:8] + '-' + phone[8:]
    
    return phone

Now that we have our html sliced into much easier to read bits, we can now go through it and find the information we are looking for. Using the patterns initialized above, we can find names, phone numbers, emails, and office information. 

It looks like recently some faculty members are no longer in the Faculty and Staff Directory and their profile pages no longer contain their contact information. The `<span>` named `facultyprofilecontainer` was removed from those pages, so if the html did not contain that `<span>`, I used a `continue` statement to skip to the next faculty member.

Once every url was visited, the faculty member's information was stored in a list in the format 
`list([name,office,phone,email],[...],...)`.

In [19]:
for url in pharmDirectoryURLS_list:
    faculty_HTML = requests.get(url[0]).text.replace('>','>\n')
    if name_start in faculty_HTML:
        nameInfo = faculty_HTML.split(name_start)[1].split(name_end)[0] 
    else:
        continue
    contactInfo = faculty_HTML.split(contactInfo_start)[1].split(contactInfo_end)[0]
    
    name = re.findall(namePattern,nameInfo)
    if len(name) > 0:
        name = name[0]
    else:
        name = 'No Name'
        
    address = re.findall(addressPattern,contactInfo)
    if len(address) > 0:
        address = address[0].replace('<br/>','')
        if address == '':
            address = 'No Address'
    else:
        address = 'No Address'
        
    phone = re.findall(phonePattern,contactInfo)
    if len(phone) > 0:
        phone = phone[0]
        phone = formatPhoneNumber(phone)
        if phone == '':
            phone = "No Phone Number"
    else:
        phone = 'No Phone Number'
        
    email = re.findall(emailPattern,contactInfo)
    if len(email) > 0:
        email = email[0]
    else:
        email = 'No Email'
    
    facultyInfo.append([name,address,phone,email])

# Print Out Information for Faculty Members 

In [20]:
for info in facultyInfo:
    print('-'*30)
    for f in info:
        print(f)

------------------------------
Kimberly Adams
117 Pharmacy Building
** South Campus **
Buffalo, NY 
(716) 645-4833
kadams4@buffalo.edu
------------------------------
Maureen A. Adolf
453 Pharmacy Building
Buffalo, NY 14214-8033 
(716) 645-6112
adolf@buffalo.edu
------------------------------
Manuj Ahuja
No Address
No Phone Number
manujahu@buffalo.edu
------------------------------
Nicole P. Albanese
209 Pharmacy Building
Buffalo, NY 14214-8033 
(716) 645-3915
npaolini@buffalo.edu
------------------------------
Kristopher M. Attwood
No Address
No Phone Number
attwood3@buffalo.edu
------------------------------
Joseph T. Balthasar
126 Cooke Hall
** North Campus **
Buffalo, NY 
(716) 645-2825
jbalthas@buffalo.edu
------------------------------
Joseph P. Balthasar
452 Pharmacy Building
Buffalo, NY 14214-8033 
(716) 645-4807
jb@buffalo.edu
------------------------------
Sathy V. Balu-Iyer
359 Pharmacy Building
Buffalo, NY 14214-8033 
(716) 645-4836
svb@buffalo.edu
--------------------------

# Extra Credit 

In [21]:
import pandas as pd

In [22]:
faculty_df = pd.DataFrame()

In [23]:
names_list = [name[0] for name in facultyInfo]
office_list = [office[1] for office in facultyInfo]
phone_list = [phone[2] for phone in facultyInfo]
email_list = [email[3] for email in facultyInfo]

In [24]:
faculty_df = pd.DataFrame(list(zip(names_list,office_list,phone_list,email_list)),
                         columns=['Name','Office','Phone Number','Email'])

In [25]:
faculty_df

,Name,Office,Phone Number,Email
0,Kimberly Adams,117 Pharmacy Building\n** South Campus **\nBuf...,(716) 645-4833,kadams4@buffalo.edu
1,Maureen A. Adolf,"453 Pharmacy Building\nBuffalo, NY 14214-8033",(716) 645-6112,adolf@buffalo.edu
2,Manuj Ahuja,No Address,No Phone Number,manujahu@buffalo.edu
3,Nicole P. Albanese,"209 Pharmacy Building\nBuffalo, NY 14214-8033",(716) 645-3915,npaolini@buffalo.edu
4,Kristopher M. Attwood,No Address,No Phone Number,attwood3@buffalo.edu
...,...,...,...,...
180,Leigh T. Yates,"1114 Kimball Hall, South Campus\nBuffalo, NY",(716) 645-3432,ltyates@buffalo.edu
181,Youngjae You,"630 Pharmacy Building\nBuffalo, NY 14214-8033",(716) 645-4843,yjyou@buffalo.edu
182,Kimberly T. Zammit,"100 High St\nBuffalo, NY 14203",(716) 859-1887,zammit@buffalo.edu
183,Congyu Zhang,No Address,No Phone Number,congyuzh@buffalo.edu


I left these bottom two cells commented out in case they were not needed.

In [26]:
#faculty_df.to_csv('ncolanProject8.csv')

In [27]:
#faculty_df.to_excel('ncolanProject8.xlsx')